In [1]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

selected_ticker = 'AAPL'
selected_ticker = 'AAPL'
start_date_str = 'August 25, 2018'
end_date_str = 'August 24, 2023'

# Convert to YYYY-MM-DD format
start_date = datetime.strptime(start_date_str, '%B %d, %Y').strftime('%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%B %d, %Y').strftime('%Y-%m-%d')

print(start_date)  # Outputs: 2018-08-25
print(end_date)    # Outputs: 2023-08-24
stock_data = yf.download(selected_ticker, start=start_date, end=end_date)

# Create a DataFrame with selected price data
stock_df = ()
stock_df = pd.DataFrame()
stock_df['High'] = stock_data['High']
stock_df['Low'] = stock_data['Low']
stock_df['Open'] = stock_data['Open']
stock_df['Close'] = stock_data['Close']

short_sma_window = 5
long_sma_window = 15



2018-08-25
2023-08-24
[*********************100%***********************]  1 of 1 completed


In [2]:
def calculate_sma(stock_data, window):
    return stock_data['Close'].rolling(window=window).mean()

# Calculate and plot indicators
def plot_sma(short_sma_window, long_sma_window, stock_data):
        # Calculate moving averages
        moving_averages = []
        short_sma_window = int(short_sma_window)
        long_sma_window = int(long_sma_window)

        if short_sma_window > 0 and long_sma_window > 0:
            short_sma = calculate_sma(stock_data, short_sma_window)
            long_sma = calculate_sma(stock_data, long_sma_window)
            moving_averages.append((f'SMA {short_sma_window}', sma_short))
            moving_averages.append((f'SMA {long_sma_window}', sma_long))

       # Plot indicators
        plt.figure(figsize=(10,6))
        plt.plot(stock_data['Close'], label='Close Price', color='blue')

        for label, ma in moving_averages:
            plt.plot(ma, label=label)

        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.title(f"{selected_ticker} Closing Price and Simple Moving Averages")
        plt.legend()
        st.pyplot(plt)
    

In [4]:
def backtest_sma_crossover_strategy(stock_df, short_sma_window, long_sma_window):
    short_sma = calculate_sma(stock_data, short_sma_window)
    long_sma = calculate_sma(stock_data, long_sma_window)

    # Buy signal: when short EMA crosses above long EMA
    stock_df['SMA Buy Signal'] = (short_sma > long_sma) & (short_sma.shift(1) <= long_sma.shift(1))

    # Sell signal: when short EMA crosses below long EMA
    stock_df['SMA Sell Signal'] = (short_sma < long_sma) & (short_sma.shift(1) >= long_sma.shift(1))

    # Assume starting with cash, so the first signal should be a buy
    initial_position = "cash"
    positions = [initial_position]

    for i in range(1, len(stock_df)):
        if stock_df['SMA Buy Signal'].iloc[i]:
            positions.append("stock")
        elif stock_df['SMA Sell Signal'].iloc[i]:
            positions.append("cash")
        else:
            positions.append(positions[-1])

    stock_df['SMA Position'] = positions
    
    # Calculate returns
    stock_df['Daily SMA Strategy Return'] = np.where(stock_df['SMA Position'] == "stock", stock_df['Close'].pct_change(), 0)
    total_sma_strategy_return = (stock_df['Daily SMA Strategy Return'] + 1).prod()
    return total_sma_strategy_return


total_sma_strategy_return = backtest_sma_crossover_strategy(stock_df, short_sma_window, long_sma_window)
print(stock_df)
print(total_sma_strategy_return)

# print(short_sma)
# print(long_sma)

                  High         Low        Open       Close  SMA Buy Signal  \
Date                                                                         
2018-08-27   54.685001   54.082500   54.287498   54.485001           False   
2018-08-28   55.134998   54.730000   54.752499   54.924999           False   
2018-08-29   55.872501   54.852501   55.037498   55.744999           False   
2018-08-30   57.064999   55.599998   55.812500   56.257500           False   
2018-08-31   57.217499   56.500000   56.627499   56.907501           False   
...                ...         ...         ...         ...             ...   
2023-08-17  177.509995  173.479996  177.139999  174.000000           False   
2023-08-18  175.100006  171.960007  172.300003  174.490005           False   
2023-08-21  176.130005  173.740005  175.070007  175.839996           False   
2023-08-22  177.679993  176.250000  177.059998  177.229996           False   
2023-08-23  181.550003  178.330002  178.520004  181.119995      